
# Table of Contents



Initialize Python session with imports.



In [1]:
from bitarray import bitarray
from datetime import date, timedelta
import numpy as np
from math import factorial
import scipy.special
import timeit

## 11 Largest product in a grid



### Q:



In the 20×20 grid below, four numbers along a diagonal line have been marked in
<del>red</del>.



In [1]:
A = np.matrix(
    " 8  2 22 97 38 15  0 40  0 75  4  5  7 78 52 12 50 77 91  8;\
     49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48  4 56 62  0;\
     81 49 31 73 55 79 14 29 93 71 40 67 53 88 30  3 49 13 36 65;\
     52 70 95 23  4 60 11 42 69 24 68 56  1 32 56 71 37  2 36 91;\
     22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80;\
     24 47 32 60 99  3 45  2 44 75 33 53 78 36 84 20 35 17 12 50;\
     32 98 81 28 64 23 67 10 26 38 40 67 59 54 70 66 18 38 64 70;\
     67 26 20 68  2 62 12 20 95 63 94 39 63  8 40 91 66 49 94 21;\
     24 55 58  5 66 73 99 26 97 17 78 78 96 83 14 88 34 89 63 72;\
     21 36 23  9 75  0 76 44 20 45 35 14  0 61 33 97 34 31 33 95;\
     78 17 53 28 22 75 31 67 15 94  3 80  4 62 16 14  9 53 56 92;\
     16 39  5 42 96 35 31 47 55 58 88 24  0 17 54 24 36 29 85 57;\
     86 56  0 48 35 71 89  7  5 44 44 37 44 60 21 58 51 54 17 58;\
     19 80 81 68  5 94 47 69 28 73 92 13 86 52 17 77  4 89 55 40;\
      4 52  8 83 97 35 99 16  7 97 57 32 16 26 26 79 33 27 98 66;\
     88 36 68 87 57 62 20 72  3 46 33 67 46 55 12 32 63 93 53 69;\
      4 42 16 73 38 25 39 11 24 94 72 18  8 46 29 32 40 62 76 36;\
     20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74  4 36 16;\
     20 73 35 29 78 31 90  1 74 31 49 71 48 86 81 16 23 57  5 54;\
      1 70 54 71 83 51 54 69 16 92 33 48 61 43 52  1 89 19 67 48",
    dtype=int)

The product of these numbers is 26 × 63 × 78 × 14 = 1788696.

What is the greatest product of four adjacent numbers in the same direction (up,
down, left, right, or diagonally) in the 20×20 grid?



### A:



Consider $A\in\mathbb{R}^{n\times m}$:

$$ A = \begin{bmatrix}{}
       a_{1,1} & a_{1,2} & \dots  & a_{1,m} \\
       a_{2,1} & a_{2,2} & \dots  & a_{2,m} \\
       \vdots  & \vdots  & \ddots & \vdots \\
       a_{n,1} & a_{n,2} & \dots  & a_{n,m}
       \end{bmatrix} $$

For a given window size $w$, define matrices for each of the four unique
directions of adjacency:

1.  $\mathbf{V}^w \in\mathbb{R}^{(n-w) \times  m   }$ for the vertical products,
2.  $\mathbf{H}^w \in\mathbb{R}^{ n    \times (m-w)}$ for the horizontal products,
3.  $\mathbf{B}^w \in\mathbb{R}^{(n-w) \times (m-w)}$ for the \`\\' (backslash-like) products,
4.  $\mathbf{F}^w \in\mathbb{R}^{(n-w) \times (m-w)}$ for the \`/' (frontslash-like) products.

where the elements of these matrices are

\begin{eqnarray*}
v^w_{i,j} & = & \prod_{k=0}^{w-1} a_{i+k, j    } \\
h^w_{i,j} & = & \prod_{k=0}^{w-1} a_{i,   j+k  } \\
b^w_{i,j} & = & \prod_{k=0}^{w-1} a_{i+k, j+k  } \\
f^w_{i,j} & = & \prod_{k=0}^{w-1} a_{i+k, j+w-k}
\end{eqnarray*}

And construct $\mathbf{V,H,B,F}$ with `numpy`:



In [1]:
def adjacent_products(A,w):
    """Find all the adjacent products in matrix ``A`` of window size ``w``."""
    n, m = A.shape
    V = np.ones((n-w,m),dtype=int)
    H = np.ones((n,m-w),dtype=int)
    B = np.ones((n-w,m-w),dtype=int)
    F = np.ones((n-w,m-w),dtype=int)
    for k in range(0,w):
        V = np.multiply(V,A[k:k-w,:])
        H = np.multiply(H,A[:,k:k-w])
        B = np.multiply(B,A[k:k-w,k:k-w])
        F = np.multiply(F,A[k:k-w,w-(k+1):-(k+1)])
    return V, H, B, F

V, H, B, F = adjacent_products(A,4)
assert B[6,8] == 1788696

`numpy` makes this task fast since we can code with matrix operations. Python
loops are surprisingly slow compared to `numpy`, as the core `numpy` operations
are actually wrappers for compiled `C`.



In [1]:
print(max([np.max(A) for A in [V,H,B,F]]))
%timeit max([np.max(A) for A in adjacent_products(A,4)])

70600674
107 µs ± 617 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

## 12 Highly divisible triangular number



### Q:



The sequence of triangle numbers is generated by adding the natural
numbers. So the 7th triangle number would be 1 + 2 + 3 + 4 + 5 + 6 + 7 = 28. The
first ten terms would be:

1, 3, 6, 10, 15, 21, 28, 36, 45, 55, &#x2026;

Let us list the factors of the first seven triangle numbers:

| $T_i$|$factors(T_i)$|
|---|---|
| 1|1|
| 3|1,3|
| 6|1,2,3,6|
| 10|1,2,5,10|
| 15|1,3,5,15|
| 21|1,3,7,21|
| 28|1,2,4,7,14,28|

We can see that 28 is the first triangle number to have over five divisors.

What is the value of the first triangle number to have over five hundred
divisors?



### A:



Getting correct results for this problem is not hard, but getting fast times is.
Before this algorithm, I tried trial division and prime factorization
implementations, but both of them took far too long to be worth sharing. The
algorithm I settled on leverages list slicing for sieving the number of divisors
and the formula for the $n$th triangular number, $T_n$.

We first need some way to sieve for the number of divisors. This is actually a
very simple function that wasn't obvious until I found it on [a different blog](https://blog.dreamshire.com/project-euler-12-solution/). I
improved it by adding the list slicing. We simply initiate a `numpy.array` of
length $n$, iterate through all possible divisors $1,2,\ldots,i,\ldots,n$,
and add a count to all the indices that are multiples of $i$. The result is
the sum of divisors for every number $0,1,...,n-1$. We can additionally find
proper divisors by subtracting one from `d[1:]`, which we will use on later
problems.



In [1]:
def num_divisors_sieve(n,proper=False):
    """Find the number of divisors for all nonnegative numbers less than ``n``.
    """
    d = np.zeros(n,dtype=int)
    for i in range(1,n//2):
        d[i::i] += 1
    if proper:
        d[1:n//2] += -1
    else:
        d[n//2:] += 1
    return d

print(num_divisors_sieve(10,True))
assert(num_divisors_sieve(10) == [0,1,2,2,3,2,4,2,4,3]).all()
assert(num_divisors_sieve(10,True) == [0,0,1,1,2,1,3,1,3,2]).all()

[0 0 1 1 2 1 3 1 3 2]

We need the function from [problem 1](001.md) that gave us $T_n$.



In [1]:
def triangular(n):
    """Find the ``n``th triangular number."""
    return n*(n+1)//2

assert triangular(1)==1
assert triangular(2)==3
assert triangular(100)==5050

We know that $T_n = \frac{n(n+1)}{2}$ (see [problem 1](001.md)) and that all $n,n+1$
are coprime. Therefore, we can break the problem into smaller problems. First,
write $T_n$ as the product of two coprime integers, keeping in mind that $n$
and any factors of $n+1$ are coprime, and $n+1$ and any factors of $n$ are
coprime.

$$ T_n =
    \begin{cases}
        n\cdot\frac{n+1}{2}, & \text{n is odd} \\
        \frac{n}{2}\cdot(n+1), & \text{n is even}
    \end{cases} $$

Consider two coprimes, $n,n'$. They have prime factorizations

$$ n = \prod_i p_i^{a_i} \\
   n' = \prod_j p_j^{a_j} $$

such that $p_i\neq p_j \,\forall\, i,j$. Therefore, the prime factorization
of $n\cdot n'$ is

$$ n\cdot n' = \left[ \prod_i p_i^{a_i} \right] \cdot
               \left[ \prod_j p_j^{a_j} \right] $$

Now, the number of divisors can be found from the prime factorization easily
using combinatorics. If we construct a divisor, $d|n$, it will have a prime
factorization

$$ d = \prod_i p_i^{b_i} $$

such that $0\leq b_i\leq a_i \,\forall\, i$. In other words, for each exponent
of $p_i$ we can choose from $0,1,...,a_i$ for the exponent in the divisor.
There are $a_i+1$ choices for every prime factor of $n$, making the total
number of unique divisors we can construct

$$ D(n) = \prod_i a_i+1 $$

Additionally, since coprimes cannot share prime factors, the number of divisors
for the product of coprimes is a multiplicative function.

$$ D(n\cdot n') = \left[ \prod_i a_i+1 \right] \cdot
                  \left[ \prod_j a_j+1 \right] = D(n)D(n') $$

We already know $T_n$ is a product of coprimes. Therefore,

$$ D(T_n) =
    \begin{cases}
        D(n)D((n+1)/2), & \text{n is odd} \\
        D(n/2)D(n+1), & \text{n is even}
    \end{cases} $$

Now we have a way to iterate over $i$ and find $D(T_i)$ without explicitly
calculating the number of divisors of $T_i$.



In [1]:
def triangular_divisors(k,limit):
    """Find the first triangular number with more than ``k`` divisors. Requires
    a limit with which to generate the sieve of number of divisors."""
    d = num_divisors_sieve(limit)
    for i in range(1,len(d)-1):
        dtri = d[i//2]*d[i+1] if i%2==0 else d[i]*d[(i+1)//2]
        if dtri>k:
            return triangular(i)
    return None

assert triangular_divisors(5,10)==28

I'll guess 15000 as the highest index our number could possibly be (it is
actually a bit less). Our algorithm takes $O(n)$ time, where $n$ is the
index of the triangular number we guess. If we had sieved over all $T_n$, this
would have taken $O(T_n)=O(n^2)$ time.



In [1]:
print(triangular_divisors(500,15000))
%timeit triangular_divisors(500,15000)

76576500
17.4 ms ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

## 13 Large sum



### Q:



Work out the first ten digits of the sum of the following one-hundred
50-digit numbers.



In [1]:
numbers = np.array([
  37107287533902102798797998220837590246510135740250,
  46376937677490009712648124896970078050417018260538,
  74324986199524741059474233309513058123726617309629,
  91942213363574161572522430563301811072406154908250,
  23067588207539346171171980310421047513778063246676,
  89261670696623633820136378418383684178734361726757,
  28112879812849979408065481931592621691275889832738,
  44274228917432520321923589422876796487670272189318,
  47451445736001306439091167216856844588711603153276,
  70386486105843025439939619828917593665686757934951,
  62176457141856560629502157223196586755079324193331,
  64906352462741904929101432445813822663347944758178,
  92575867718337217661963751590579239728245598838407,
  58203565325359399008402633568948830189458628227828,
  80181199384826282014278194139940567587151170094390,
  35398664372827112653829987240784473053190104293586,
  86515506006295864861532075273371959191420517255829,
  71693888707715466499115593487603532921714970056938,
  54370070576826684624621495650076471787294438377604,
  53282654108756828443191190634694037855217779295145,
  36123272525000296071075082563815656710885258350721,
  45876576172410976447339110607218265236877223636045,
  17423706905851860660448207621209813287860733969412,
  81142660418086830619328460811191061556940512689692,
  51934325451728388641918047049293215058642563049483,
  62467221648435076201727918039944693004732956340691,
  15732444386908125794514089057706229429197107928209,
  55037687525678773091862540744969844508330393682126,
  18336384825330154686196124348767681297534375946515,
  80386287592878490201521685554828717201219257766954,
  78182833757993103614740356856449095527097864797581,
  16726320100436897842553539920931837441497806860984,
  48403098129077791799088218795327364475675590848030,
  87086987551392711854517078544161852424320693150332,
  59959406895756536782107074926966537676326235447210,
  69793950679652694742597709739166693763042633987085,
  41052684708299085211399427365734116182760315001271,
  65378607361501080857009149939512557028198746004375,
  35829035317434717326932123578154982629742552737307,
  94953759765105305946966067683156574377167401875275,
  88902802571733229619176668713819931811048770190271,
  25267680276078003013678680992525463401061632866526,
  36270218540497705585629946580636237993140746255962,
  24074486908231174977792365466257246923322810917141,
  91430288197103288597806669760892938638285025333403,
  34413065578016127815921815005561868836468420090470,
  23053081172816430487623791969842487255036638784583,
  11487696932154902810424020138335124462181441773470,
  63783299490636259666498587618221225225512486764533,
  67720186971698544312419572409913959008952310058822,
  95548255300263520781532296796249481641953868218774,
  76085327132285723110424803456124867697064507995236,
  37774242535411291684276865538926205024910326572967,
  23701913275725675285653248258265463092207058596522,
  29798860272258331913126375147341994889534765745501,
  18495701454879288984856827726077713721403798879715,
  38298203783031473527721580348144513491373226651381,
  34829543829199918180278916522431027392251122869539,
  40957953066405232632538044100059654939159879593635,
  29746152185502371307642255121183693803580388584903,
  41698116222072977186158236678424689157993532961922,
  62467957194401269043877107275048102390895523597457,
  23189706772547915061505504953922979530901129967519,
  86188088225875314529584099251203829009407770775672,
  11306739708304724483816533873502340845647058077308,
  82959174767140363198008187129011875491310547126581,
  97623331044818386269515456334926366572897563400500,
  42846280183517070527831839425882145521227251250327,
  55121603546981200581762165212827652751691296897789,
  32238195734329339946437501907836945765883352399886,
  75506164965184775180738168837861091527357929701337,
  62177842752192623401942399639168044983993173312731,
  32924185707147349566916674687634660915035914677504,
  99518671430235219628894890102423325116913619626622,
  73267460800591547471830798392868535206946944540724,
  76841822524674417161514036427982273348055556214818,
  97142617910342598647204516893989422179826088076852,
  87783646182799346313767754307809363333018982642090,
  10848802521674670883215120185883543223812876952786,
  71329612474782464538636993009049310363619763878039,
  62184073572399794223406235393808339651327408011116,
  66627891981488087797941876876144230030984490851411,
  60661826293682836764744779239180335110989069790714,
  85786944089552990653640447425576083659976645795096,
  66024396409905389607120198219976047599490197230297,
  64913982680032973156037120041377903785566085089252,
  16730939319872750275468906903707539413042652315011,
  94809377245048795150954100921645863754710598436791,
  78639167021187492431995700641917969777599028300699,
  15368713711936614952811305876380278410754449733078,
  40789923115535562561142322423255033685442488917353,
  44889911501440648020369068063960672322193204149535,
  41503128880339536053299340368006977710650566631954,
  81234880673210146739058568557934581403627822703280,
  82616570773948327592232845941706525094512325230608,
  22918802058777319719839450180888072429661980811197,
  77158542502016545090413245809786882778948721859617,
  72107838435069186155435662884062257473692284509516,
  20849603980134001723930671666823555245252804609722,
  53503534226472524250874054075591789781264330331690
])

### A:



Python has support for large integers, making this problem trivial.



In [1]:
S = sum(numbers)
print(str(S)[:10])

%timeit str(sum(numbers))[:10]y

5537376230
4.4 µs ± 62 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

## 14 Longest Collatz sequence



### Q:



The following iterative sequence is defined for the set of positive integers:

n → n/2 (n is even)
n → 3n + 1 (n is odd)

Using the rule above and starting with 13, we generate the following sequence:

13 → 40 → 20 → 10 → 5 → 16 → 8 → 4 → 2 → 1
It can be seen that this sequence (starting at 13 and finishing at 1) contains
10 terms. Although it has not been proved yet (Collatz Problem), it is thought
that all starting numbers finish at 1.

Which starting number, under one million, produces the longest chain?

NOTE: Once the chain starts the terms are allowed to go above one million.



### A:



The [Collatz sequence](https://en.wikipedia.org/wiki/Collatz_conjecture) is another one of those strange unsolved problems in
mathematics. If you check out a [directed graph](https://en.wikipedia.org/wiki/Collatz_conjecture#/media/File:Collatz-graph-all-30-no27.svg) of the first few numbers, it is
easy to see how different starting numbers, while they may begin with very
different sequences, they will invariably converge on the same sequence. We can
construct a tree like this, for starting numbers $1,\ldots,n$, by caching the
results in a binary tree. This would be very easy to do, since we could build
sequences, adding new nodes at the head, and checking if the head appears in the
Collatz tree, only iterating through $O(n)$ nodes. 

All we need to solve this problem is the sequence length, however. Let's say we
have a list $c_1,c_2,\ldots,c_{n-1}$, that is length of the Collatz sequences
that start with the numbers $1,2,\ldots,n-1$. If we want to find the length of
the sequence starting with $n$, we simply have to count the number of steps,
$k$ it takes to reach some $m < n$. Then, we can store $c_n=c_m+k$ in the
list. This will save us a huge amount of time in constructing the list, since we
can only iterate over the branches of the tree, rather than the complete
sequence every time.



In [1]:
def collatz_sieve(n):
    """Find the lengths of the collatz sequences for every starting number
    ``1,2,...,n-1``.
    """
    collatz = np.zeros(n-1,dtype=int)
    collatz[0] = 1
    for i in range(1,n-1):
        c = i+1
        k = 0
        while c!=1 and c>i:
            c = c//2 if c%2==0 else 3*c+1
            k += 1
        collatz[i] = collatz[c-1] + k
    return collatz

assert(collatz_sieve(10)==[1,2,8,3,6,9,17,4,20]).all()

I also wrote a helper function to more quickly find the answer to this problem
in a generalized fashion.



In [1]:
def longest_collatz(n):
    """Find the starting number (less than ``n``) and sequence length for the
    longest collatz sequence.
    """
    c = collatz_sieve(n)
    return np.argmax(c)+1, np.amax(c)

assert longest_collatz(2) == (1, 1)
assert longest_collatz(10) == (9, 20)
assert longest_collatz(100) == (97, 119)
assert longest_collatz(1000) == (871, 179)

Now our algorithm runs in $O(n)$.



In [1]:
print(longest_collatz(1000000))
%timeit longest_collatz(1000000)

(837799, 525)
954 ms ± 6.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## 15 Lattice paths



### Q:



Starting in the top left corner of a 2×2 grid, and only being able to move to
the right and down, there are exactly 6 routes to the bottom right corner.

How many such routes are there through a 20×20 grid?



### A:



Let's generalize this for a $n\times m$ grid. We always have to go down $n$
times and right $m$ times, but we can choose the order in which to make these
moves. In other words, we are looking for the number of unique sequences of
exactly $n$ "downs" and $m$ "rights". The sequence must be $n+m$ steps,
and we will choose $n$ of those steps to use our "down" moves. The number of
ways in which we can allocate those "down" moves is just a combination of the
following form.

$$ {n+m \choose n} = {n+m \choose m} $$

Since we have an explicit formula, our calculation can be super speedy.



In [1]:
print(int(scipy.special.comb(20+20,20)))
%timeit int(scipy.special.comb(20+20,20))

137846528820
4.43 µs ± 8.18 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

## 16 Power digit sum



### Q:



$2^{15}=32768$ and the sum of its digits is $3+2+7+6+8=26$.

What is the sum of the digits of the number $2^{1000}$?



### A:



Again, Python has support for integers of arbitrary size, so this problem is
trivial.



In [1]:
print(sum([int(d) for d in str(2**1000)]))
%timeit sum([int(d) for d in str(2**1000)])

1366
41 µs ± 974 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

## 17 Number letter counts



### Q:



If the numbers 1 to 5 are written out in words: one, two, three, four, five,
then there are 3 + 3 + 5 + 4 + 4 = 19 letters used in total.

If all the numbers from 1 to 1000 (one thousand) inclusive were written out in
words, how many letters would be used?

NOTE: Do not count spaces or hyphens. For example, 342 (three hundred and
forty-two) contains 23 letters and 115 (one hundred and fifteen) contains 20
letters. The use of "and" when writing out numbers is in compliance with British
usage.



### A:



Brute force. I didn't like this problem. So I hard coded it and let it be.



In [1]:
c = np.zeros(5,dtype=int)
# 0) 1-9
ones = ['one','two','three','four','five','six','seven','eight','nine']
c[0] = len("".join(ones))

# 1) 10-19
teens = ['ten','eleven','twelve','thirteen','fourteen','fifteen','sixteen',
  'seventeen','eighteen','nineteen']
c[1] = len("".join(teens))

# 2) 20-99
tens = ['twenty','thirty','forty','fifty','sixty','seventy','eighty','ninety']
c[2] = len("".join(tens))*10 # tens place
c[2] += c[0]*len(tens)       # ones place

# 3) 100-999
c[3] = 100*(c[0]+9*len("hundred")) + 9*99*len("and") # hundreds place
c[3] += 9*(sum(c[:3])) # tens and ones place

# 4) 1000
c[4] = len("onethousand")

print(sum(c))

21124

## 18 Maximum path sum I



### Q:



By starting at the top of the triangle below and moving to adjacent numbers
on the row below, the maximum total from top to bottom is 23.

       3
      7 4
     2 4 6
    8 5 9 3

That is, 3 + 7 + 4 + 9 = 23.

Find the maximum total from top to bottom of the triangle below:

                         75
                        95 64
                      17 47 82
                     18 35 87 10
                   20 04 82 47 65
                  19 01 23 75 03 34
                88 02 77 73 07 63 67
               99 65 04 28 06 16 70 92
             41 41 26 56 83 40 80 70 33
            41 48 72 33 47 32 37 16 94 29
          53 71 44 65 25 43 91 52 97 51 14
         70 11 33 28 77 73 17 78 39 68 17 57
       91 71 52 38 17 14 91 43 58 50 27 29 48
      63 66 04 68 89 53 67 30 73 16 69 87 40 31
    04 62 98 27 23 09 70 98 73 93 38 53 60 04 23

NOTE: As there are only 16384 routes, it is possible to solve this problem by
trying every route. However, Problem 67, is the same challenge with a triangle
containing one-hundred rows; it cannot be solved by brute force, and requires a
clever method! ;o)



### A:



First, we're going to need a way to read a triangular matrix from a file because
problem 67 requires it.



In [1]:
def read_triangle(filename):
    """Read a triangular matrix from a file or string. Uses an array of arrays
    rather than 2d array because the inner arrays aren't uniform length.
    """
    with open(filename,"r") as file:
        tri = np.array(
            [np.array(
                 [int(n) for n in line.split()]
             ) for line in file]
        )
    return tri

tri = read_triangle("018.txt")
assert tri[0][0] == 75
assert(tri[1] == [95,64]).all()
assert(tri[4] == [20,4,82,47,65]).all()

To find the maximum possible sum, we could iterate over every possible path. For
a triangle of depth $n$, each path is $n$ nodes long and there are $2^n$
possible paths. Iterating over such a large number isn't ideal, so let's
simplify it. The maximum possible path of the tree with root node, $r$, must
start towards the subtree whose maximum possible path is larger. We can start at
the bottom, taking the maximum of two leaf nodes and adding it to the parents,
and iterating up the tree until we reach the root. This is illustrated on the
four-deep triangular matrix given in the problem statement.

       3           3         3      23
      7 4   ->   7  4   -> 20 19 -> 
     2 4 6     10 13 15
    8 5 9 3



In [1]:
def max_sum_triangle(tri):
    """Find the maximum sum path in a binary tree. Requires tree in a matrix
    form. Uses an array of arrays rather than 2d array because the inner arrays
    aren't uniform length.
    """
    for i in range(len(tri)-1,0,-1):
        tri[i-1] += np.maximum(tri[i][1:],tri[i][:-1])
    return tri[0][0]

tri_test = [[3],[7,4],[2,4,6],[8,5,9,3]]
assert max_sum_triangle(tri_test) == 23

With this idea, we have to iterate over all the non-leaf nodes, or $T_{n-1}$
nodes (where $T_n$ is the $n$th triangular number), which is $O(n^2)$ with
respect to tree depth.



In [1]:
tri = read_triangle("018.txt")
print(max_sum_triangle(tri))

%timeit max_sum_triangle(read_triangle("018.txt"))

1074
89.4 µs ± 748 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

## 19 Counting Sundays



### Q



You are given the following information, but you may prefer to do some
research for yourself.

-   1 Jan 1900 was a Monday.
-   Thirty days has September,
    April, June and November.
    All the rest have thirty-one,
    Saving February alone,
    Which has twenty-eight, rain or shine.
    And on leap years, twenty-nine.
-   A leap year occurs on any year evenly divisible by 4, but not on a century
    unless it is divisible by 400.

How many Sundays fell on the first of the month during the twentieth century
(1 Jan 1901 to 31 Dec 2000)?



### A:



I could code this up explicitly, but it is much easier to use built-in `date`
object. The days of week are coded `0-6` for Monday to Sunday. Brute force by
looping over all the 1st of months until we run out of bounds.



In [1]:
def pe019(years,day,weekday):
    """Count the number of times a given day of the month falls on a day of the
    week for the given years. Weekdays are coded ``0-6`` for Mon through Sun.
    """
    count = 0
    for year in years:
        for month in range(1,13):
            if date(year,month,day).weekday()==weekday:
                count += 1
    return count

assert pe019([2019],1,6)==2

This is surprisingly slow, but manageable.



In [1]:
print(pe019(range(1901,2001),1,6))
%timeit pe019(range(1901,2001),1,6)

171
252 µs ± 5.88 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

## 20 Factorial digit sum



### Q:



n! means n × (n − 1) × &#x2026; × 3 × 2 × 1

For example, 10! = 10 × 9 × &#x2026; × 3 × 2 × 1 = 3628800,
and the sum of the digits in the number 10! is 3 + 6 + 2 + 8 + 8 + 0 + 0 = 27.

Find the sum of the digits in the number 100!



### A:



Trivial because Python can handle large integers.



In [1]:
def factsum(n):
    """Find the sum of the digits of ``n!``."""
    fact = factorial(n)
    return sum(map(int, str(fact)))

assert factsum(1) == 1
assert factsum(2) == 2
assert factsum(3) == 6
assert factsum(4) == 6
assert factsum(10) == 27

In [1]:
print(factsum(100))
%timeit factsum(100)

648
15.2 µs ± 541 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)